<a href="https://colab.research.google.com/github/SauloHenriqueAguiar/backtestBacktrading/blob/main/AverageTrueRangebacktesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yahoo_fin

     |████████████████████████████████| 81 kB 3.4 MB/s 
     |████████████████████████████████| 83 kB 2.4 MB/s 
     |████████████████████████████████| 103 kB 43.6 MB/s 
     |████████████████████████████████| 127 kB 42.6 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=a06ba5fada94bcd8596301e8d92e3d632054694946a745426a2c098cb896e397
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=d878142c97fff499446abe345b5aead7724e9fb75fde767e3b684eebbfd599f0
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=559a2c8dc5fd714a62dee6987e078ce8a31d439c14befe8647abdcf133dbcd0f
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36ce

In [4]:
#instala biblioteca yfinance
!pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 5.3 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=5caf432e2229330898f6bc48eae3cb0de4cced50ed399844f998ee7af558209f
  Stored in directory: /tmp/pip-ephem-wheel-cache-w3u0dura/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [5]:
#instala request
!pip install requests_html

In [6]:
# Instalar Backtrader
!pip install backtrader

     |████████████████████████████████| 410 kB 4.1 MB/s 


In [44]:
!pip install backtrader[plotting]

In [34]:
# Obtenha os pacotes necessários

from datetime import datetime
import backtrader as bt
from IPython.display import display, Image
import yfinance as yf

In [35]:
from yahoo_fin.stock_info import get_data
bitcoin_weekly = get_data("BTC-USD", start_date="12/04/2020", end_date="03/11/2021", index_as_date = True, interval="1d")
bitcoin_weekly

,open,high,low,close,adjclose,volume,ticker
2020-12-04,19446.966797,19511.404297,18697.193359,18699.765625,18699.765625,33872388058,BTC-USD
2020-12-05,18698.384766,19160.449219,18590.193359,19154.230469,19154.230469,27242455064,BTC-USD
2020-12-06,19154.179688,19390.500000,18897.894531,19345.121094,19345.121094,25293775714,BTC-USD
2020-12-07,19343.128906,19411.828125,18931.142578,19191.630859,19191.630859,26896357742,BTC-USD
2020-12-08,19191.529297,19283.478516,18269.945312,18321.144531,18321.144531,31692288756,BTC-USD
...,...,...,...,...,...,...,...
2021-03-07,48918.679688,51384.367188,48918.679688,51206.691406,51206.691406,43137459378,BTC-USD
2021-03-08,51174.117188,52314.070312,49506.054688,52246.523438,52246.523438,48597428048,BTC-USD
2021-03-09,52272.968750,54824.117188,51981.832031,54824.117188,54824.117188,50912227385,BTC-USD
2021-03-10,54824.011719,57258.253906,53290.890625,56008.550781,56008.550781,57295577614,BTC-USD


In [36]:
bitcoin_weekly.to_csv('/content/bitcoin.csv')


In [80]:
# Instantiate Cerebro engine
cerebro = bt.Cerebro()


In [57]:
#Add data feed to Cerebro
data = bt.feeds.YahooFinanceCSVData(dataname='bitcoin.csv')
cerebro.adddata(data)


In [72]:
import backtrader as bt

class AverageTrueRange(bt.Strategy):
  
	def log(self, txt, dt=None):
		dt = dt or self.datas[0].datetime.date(0)
		print(f'{dt.isoformat()} {txt}') #Print date and close
		
	def __init__(self):
		self.dataclose = self.datas[0].close
		self.datahigh = self.datas[0].high
		self.datalow = self.datas[0].low
		
	def next(self):
		range_total = 0
		for i in range(-13, 1):
			true_range = self.datahigh[i] - self.datalow[i]
			range_total += true_range
		ATR = range_total / 14

		self.log(f'Close: {self.dataclose[0]:.2f}, ATR: {ATR:.4f}')

In [81]:
# Default position size
cerebro.addsizer(bt.sizers.SizerFix, stake=3)

In [82]:
# Add strategy to Cerebro
cerebro.addstrategy(AverageTrueRange)


0

In [83]:
if __name__ == '__main__':
    # Run Cerebro Engine
    start_portfolio_value = cerebro.broker.getvalue()

    cerebro.run()

    end_portfolio_value = cerebro.broker.getvalue()
    pnl = end_portfolio_value - start_portfolio_value
    print(f'Starting Portfolio Value: {start_portfolio_value:2f}')
    print(f'Final Portfolio Value: {end_portfolio_value:2f}')
    print(f'PnL: {pnl:.2f}')

Starting Portfolio Value: 10000.000000
Final Portfolio Value: 10000.000000
PnL: 0.00
